Test Phase4_3d file

In [1]:
# install anaconda: https://www.anaconda.com/products/individual
# if use Visual Studio Code, install python extension and select the python interpreter that includes anaconda
# install micropip: https://pypi.org/project/micropip/. You can also run anaconda prompt and type "pip install micropip"
import numpy as np
import micropip
await micropip.install("scipy")

In [2]:
from Phase4_trace_3d import Phase4_trace_3d
# from generateTestPositions import generateTestPositions
import generateTestPositions
from generateTestPositions import get_velocity_function
from generateTestPositions import generateTestPositions
from generateTestPositions import get_initial_position
# Cell 4
# Testing with dummy data
NumOfDataPoints = 2
clusterness = 0.1 # smaller number the more clustered
# Input conditions
initial_positions = []
v = []
for i in range(NumOfDataPoints):
    [x,y,z] = get_initial_position(i)
    initial_positions.append([x, y, z])
    vel = get_velocity_function(i)
    v.append(vel)
# initial_positions = np.zeros((NumOfDataPoints,3))
# initial_positions[0] = get_initial_position(1)
# v.append(lambda t: [np.sin(t), np.cos(t), 1])
noise = 1e-3
theta_degrees = 1.8
rev = 2  # revolutions of camera for the entire process
NOS = int(rev * 360 / theta_degrees)
NOS_per_section = 180  # must be larger than 5 to satisfy equations
camera_speed = 0.5  # in Hz or revolution per second
SRD = 1  # m, Source-Reference Distance
RDD = 1  # m, Reference-Detector (screen) Distance
radius = 1

method = 'acceleration'
dataPiling = 'overlap'

# Auto-calculations of the rest of the parameters derived from the setting above
delta_T = camera_speed * theta_degrees / 360
shots_per_second = 1 / delta_T

# Define the velocity function
# v = lambda t: [0.9 * np.sin(t), 0.9 * np.cos(t), 1]

# Pack conditions into a list
conditions = [noise, delta_T, NOS, theta_degrees, NOS_per_section, SRD, RDD,method,dataPiling]

xz_proj = np.zeros((NOS, NumOfDataPoints*2))
real_positions = np.zeros((NOS, NumOfDataPoints*3))
# Generate test positions
for i in range(NumOfDataPoints):
    vel = v[i]
    xz_proj[:,i*2:i*2+2], real_positions[:,i*3:i*3+3]= generateTestPositions(vel, initial_positions[i], conditions)

In [3]:
# Print the first few rows of the generated data
print("xz_proj:")
print(xz_proj[:])
print("real_positions:")
print(real_positions[:])

xz_proj:
[[-4.68026559e-04 -1.49641352e-03  3.99799037e-01 -5.73251410e-04]
 [ 1.08528531e-03 -9.67513962e-05  4.02117965e-01 -1.23034447e-03]
 [ 1.10175134e-03  1.59598319e-04  4.03507269e-01  5.53450987e-05]
 ...
 [ 1.20399768e-03  1.27215093e-01  3.88714336e-01 -1.24810170e-01]
 [ 2.06841215e-03  1.27628158e-01  3.93668296e-01 -1.25660678e-01]
 [-2.25854726e-04  1.26642828e-01  3.97726051e-01 -1.26184408e-01]]
real_positions:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.00000000e-01
   0.00000000e+00  0.00000000e+00]
 [ 2.49997430e-04  0.00000000e+00  9.81742658e-07  1.99750003e-01
   0.00000000e+00 -9.81742658e-07]
 [ 4.99979439e-04  0.00000000e+00  3.92691007e-06  1.99500021e-01
   0.00000000e+00 -3.92691007e-06]
 ...
 [ 7.49930606e-04  0.00000000e+00  6.36531419e-02  1.99250069e-01
   0.00000000e+00 -6.36531419e-02]
 [ 4.99979439e-04  0.00000000e+00  6.36580503e-02  1.99500021e-01
   0.00000000e+00 -6.36580503e-02]
 [ 2.49997430e-04  0.00000000e+00  6.36609955e-02  1.9975

In [4]:
# Import the function directly from the module
from Phase4_trace_3d import Phase4_trace_3d
from smooth_points import smooth_points
import particle_path_finder
from particle_path_finder import track_particles

# Ai recognition caller
alpha = np.radians(theta_degrees)  # Example rotation angle in radians
tracked = track_particles(xz_proj, alpha)
print(tracked)

estimated_positions = np.zeros((NOS,3*NumOfDataPoints))
# Then you can call it just by its name:
for i in range(NumOfDataPoints):
    estimated_positions_single = Phase4_trace_3d(conditions, xz_proj[:,i*2:i*2+2])
    estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

    estimated_positions[:,i*3:i*3+3] = estimated_positions_single



[[-4.68026559e-04 -1.49641352e-03  3.99799037e-01 -5.73251410e-04]
 [ 1.08528531e-03 -9.67513962e-05  4.02117965e-01 -1.23034447e-03]
 [ 1.10175134e-03  1.59598319e-04  4.03507269e-01  5.53450987e-05]
 ...
 [ 3.88714336e-01 -1.24810170e-01  1.20399768e-03  1.27215093e-01]
 [ 3.93668296e-01 -1.25660678e-01  2.06841215e-03  1.27628158e-01]
 [ 3.97726051e-01 -1.26184408e-01 -2.25854726e-04  1.26642828e-01]]
NOS:  400
NOS_per_Section:  180
NOS:  400
NOS_per_Section:  180


In [5]:
print('estimated positions: \n',estimated_positions)
# print('real positions: \n',real_positions)

estimated positions: 
 [[-2.54556473e-04 -5.13470931e-04 -8.82033772e-04  1.95052315e-01
  -4.87826622e-03  7.73184495e-04]
 [ 1.89847287e-05 -5.25703041e-04 -8.57723264e-04  1.94886754e-01
  -4.89171401e-03  7.51254373e-04]
 [ 2.91517346e-04 -5.37799179e-04 -8.31971162e-04  1.94721250e-01
  -4.90504976e-03  7.27862418e-04]
 ...
 [ 1.06433018e-03  1.60662960e-03  6.11084464e-02  1.89402179e-01
  -7.01245862e-03 -6.10427447e-02]
 [ 7.81267776e-04  1.65910297e-03  6.10699971e-02  1.89470594e-01
  -7.04607822e-03 -6.10049238e-02]
 [ 4.96924602e-04  1.71205332e-03  6.10294849e-02  1.89538259e-01
  -7.07999520e-03 -6.09650504e-02]]


In [6]:
import matplotlib
print(matplotlib.get_backend())
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')  # or another interactive backend

# from mpl_toolkits.mplot3d import Axes3D
import os


def Phase4Graph(real_positions, estimated_positions, conditions, v, f2):
    noise, delta_T, NOS, theta_degree, N, SRD, RDD, method, dataPiling = conditions
    
    plt.ion()
    # Ensure that real_positions and estimated_positions are the same length
    real_positions = real_positions[:len(estimated_positions)]
    
    fig = plt.figure(f2, figsize=(20, 16), dpi=100)
    
    ax1 = fig.add_subplot(111, projection='3d')
    
    for i in range(NumOfDataPoints):
        ax1.plot(real_positions[:, i*3], real_positions[:, i*3+1], real_positions[:, i*3+2], 'r', markersize=10)
        ax1.plot(estimated_positions[:, i*3], estimated_positions[:, i*3+1], estimated_positions[:, i*3+2], 'b', markersize=20)
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_xlim([-0.5, 0.5])
    ax1.set_ylim([-0.5, 0.5])
    ax1.set_zlim([0, 1.5])
    ax1.set_title('Real Positions vs Estimated Positions in 3D')
    ax1.legend(['Real Positions', 'Estimated Positions'])
    
    distances = np.linalg.norm(real_positions - estimated_positions, axis=1)
    min_val, min_idx = np.min(distances), np.argmin(distances)
    max_val, max_idx = np.max(distances), np.argmax(distances)
    
    # ax1.text(*real_positions[min_idx], f'Min ({min_idx}): {real_positions[min_idx]}', color='r')
    # ax1.text(*estimated_positions[min_idx], f'Min ({min_idx}): {estimated_positions[min_idx]}', color='b')
    # ax1.text(*real_positions[max_idx], f'Max ({max_idx}): {real_positions[max_idx]}', color='r')
    # ax1.text(*estimated_positions[max_idx], f'Max ({max_idx}): {estimated_positions[max_idx]}', color='b')
    
    print(f'Noise: {noise}\nMin Distance: {min_val}\nMax Distance: {max_val}\nNumber of Shots: {NOS}\nRotation: {theta_degree}\nTime Between Shots: {delta_T}\nNOS Per Section: {N}\nMethod: {method}\nData Piling: {dataPiling}')
    
    v_str = str(v).replace('<function ', '').replace(' at 0x' + hex(id(v))[2:] + '>', '')
    
    print(f'True velocity:\n{v_str}\nt is time elapsed')
    
    
    plt.show()


module://matplotlib_inline.backend_inline


In [7]:
f2 = 0
Phase4Graph(real_positions, estimated_positions, conditions, v, f2)

Noise: 0.001
Min Distance: 0.0052120025586055215
Max Distance: 0.013099278965137016
Number of Shots: 400
Rotation: 1.8
Time Between Shots: 0.0025
NOS Per Section: 180
Method: acceleration
Data Piling: overlap
True velocity:
[get_velocity_function.<locals>.<lambda> at 0x00000185801FDE40>, get_velocity_function.<locals>.<lambda> at 0x0000018581AAC540>]
t is time elapsed
